In [14]:
from mathutils import Vector
import pandas as pd

In [15]:
def sec_to_index(sec):
    return sec * 1000 // 33

## Extract columns from dataset

In [ ]:
dataset_columns = ['8 Acc LUA^ accX', '9 Acc LUA^ accY', '10 Acc LUA^ accZ']
# dataset_columns = ['14 Acc LH accX', '15 Acc LH accY', '16 Acc LH accZ']

csv_dataset_path = 'E:/University/Courses/CNG 492/workspace/scripts/S1-ADL1_sensors_data.txt.new.csv'
df_dataset = pd.read_csv(csv_dataset_path)
vectors_dataset = [Vector(row[dataset_columns].tolist()) for _, row in df_dataset.iterrows()]

# Save vectors_dataset into a csv file for later use
df_rows = pd.DataFrame(df_dataset, columns=dataset_columns)
vectors_dataset_df = pd.DataFrame(df_rows)
# vectors_dataset_df.to_csv('E:/University/Courses/CNG 492/workspace/scripts/arduino_output/vectors_dataset.csv', index=False)


In [58]:
# Specify the path to the CSV file
# You can use any file here for applying the transformation, but the data you use to calculate the transformation should be from a static period 
csv_path = 'E:/University/Courses/CNG 492/workspace/scripts/arduino_output/arduino_data_worldacc_20240502_072215.csv'
# csv_path = 'E:/University/Courses/CNG 492/workspace/scripts/arduino_output/arduino_data_worldacc_LUA_Lie_20240502_112414.csv'

vectors_dataset_csv_path = 'E:/University/Courses/CNG 492/workspace/scripts/arduino_output/vectors_dataset.csv'

In [59]:
# Read the CSV file into a dataframe
df = pd.read_csv(csv_path)
# vectors_dataset_df = pd.read_csv(vectors_dataset_csv_path)

vectors = [Vector(row.tolist()[1:]) for _, row in df.iterrows()]
vectors_dataset = [Vector(row.tolist()) for _, row in vectors_dataset_df.iterrows()]

In [ ]:
print(vectors[:10])

In [52]:
# IMPORTANT: Update intervals here according to the data you have
# Interval: 0s to 1s for vectors, 18s to 19s for vectors_dataset

vecs1 = []
vecs2 = []
# quats = []
for i in range(1000 // 33):
    v1 = vectors[sec_to_index(0) + i]
    v2 = vectors_dataset[sec_to_index(18) + i]

    vecs1.append(v1)
    vecs2.append(v2)

    # # Rotation difference between v1 and v2
    # q1 = v1.rotation_difference(v2)

    # quats.append(q1)

print('v1:', vecs1[:10])
print('v2:', vecs2[:10])

vm1 = sum(vecs1, Vector()) / len(vecs1)
vm2 = sum(vecs2, Vector()) / len(vecs2)

print(f'v1 mean: {vm1}')
print(f'v2 mean: {vm2}')

qm = vm1.normalized().rotation_difference(vm2.normalized())
print(qm)

v1: [Vector((3.1600000858306885, 20.1299991607666, 0.6700000166893005)), Vector((3.109999895095825, 17.31999969482422, -3.4000000953674316)), Vector((3.640000104904175, 21.389999389648438, 0.6000000238418579)), Vector((3.740000009536743, 24.329999923706055, 0.9900000095367432)), Vector((5.199999809265137, 26.18000030517578, 2.380000114440918)), Vector((4.860000133514404, 27.790000915527344, 2.2699999809265137)), Vector((7.679999828338623, 28.68000030517578, 1.090000033378601)), Vector((6.929999828338623, 33.7599983215332, 3.0)), Vector((7.510000228881836, 33.0, 1.6200000047683716)), Vector((7.920000076293945, 25.510000228881836, -0.7699999809265137))]
v2: [Vector((109.0, 997.0, 41.0)), Vector((99.0, 1005.0, 39.0)), Vector((131.0, 1013.0, 32.0)), Vector((113.0, 1052.0, 42.0)), Vector((132.0, 1025.0, 19.0)), Vector((151.0, 998.0, 10.0)), Vector((144.0, 996.0, 8.0)), Vector((156.0, 997.0, 24.0)), Vector((144.0, 1000.0, 22.0)), Vector((143.0, 1016.0, 2.0))]
v1 mean: Vector((7.4080004692077

## Rotation difference
Rotating vm1 with qm results in vm2
rotating vm2 with inverse of qm results in vm1

In [53]:
print(vm1.normalized())
print(vm2.normalized())

vx = vm2.normalized().copy()
vx.rotate(qm.inverted())
print(vx)

Vector((0.2869371175765991, 0.9551531672477722, -0.07314160466194153))
Vector((0.15028220415115356, 0.9885730743408203, 0.011771452613174915))
Vector((0.28693708777427673, 0.9551530480384827, -0.07314158976078033))


## Similar direction, but different magnitude

In [ ]:
for i in range(len(vecs1)):
    v1r = vecs1[i].copy()
    v1r.rotate(qm)
    print(f'{v1r}, {vecs2[i]}')

## Similar direction and magnitude, by scaling 

In [ ]:
vmm1 = sum([v.magnitude for v in vecs1]) / len(vecs1)
vmm2 = sum([v.magnitude for v in vecs2]) / len(vecs2)

print(vmm1)
print(vmm2)

# Now scale vecs1 to match vecs2
for i in range(len(vecs1)):
    v1r = vecs1[i].copy()
    v1r.rotate(qm)
    v1r.normalize()
    v1r *= vmm2
    print(f'{v1r}, {vecs2[i]}')


## Rotation difference after offsetting vecs2 Y axis
Subtract gravity (1000 milli g) from vecs2

In [ ]:
gravity = Vector((0, 1000, 0))
vecs2_no_g = [v - gravity for v in vecs2]

vm2_no_g = sum(vecs2_no_g, Vector()) / len(vecs2_no_g)
vmm2_no_g = sum([v.magnitude for v in vecs2_no_g]) / len(vecs2_no_g)

qm_no_g = vm1.normalized().rotation_difference(vm2_no_g.normalized())
print(f'{qm_no_g}')

# Now scale vecs1 to match vecs2
for i in range(len(vecs1)):
    v1r = vecs1[i].copy()
    v1r.rotate(qm_no_g)
    v1r.normalize()
    v1r *= vmm2_no_g

    # # No grav
    # print(f'{v1r}, {vecs2_no_g[i]}')

    # With grav
    v1r += gravity
    print(f'{v1r}, {vecs2[i]}')


## Applying to segment with moving sensor

In [ ]:
# print(df[['Axis 0']])

for i in range(len(vectors)):
    v1r = vectors[i].copy()
    v1r.rotate(qm_no_g)
    v1r.normalize()
    v1r *= vmm2_no_g

    # With grav
    v1r += gravity

    # Update in df
    df.at[i, 'Axis 0'] = round(v1r.x)
    df.at[i, 'Axis 1'] = round(v1r.y)
    df.at[i, 'Axis 2'] = round(v1r.z)

# print(df[['Axis 0']])

In [ ]:
# Save new file
df.to_csv(csv_path.replace('.csv', '_rotated.csv'), index=False)

## Translation instead of (gravity + scaling)

In [62]:
# Difference between the two means
trans_vec = vm2 - vm1
print(f'trans_vec: {trans_vec}')

sum_translated = Vector()

# Now scale vecs1 to match vecs2
for i in range(len(vecs1)):
    v1r = vecs1[i].copy()
    
    # IMPORTANT!!!!!! Rotate first then translate
    v1r.rotate(qm)
    v1r += trans_vec

    sum_translated += v1r

    # print(f'{v1r}, {vecs2[i]}')

print(f'v1 new mean: {sum_translated / len(vecs1)}')
print(f'v2 mean: {vm2}')

trans_vec: Vector((145.79200744628906, 983.1070556640625, 13.888334274291992))
v1 new mean: Vector((149.67196655273438, 1008.629638671875, 14.192244529724121))
v2 mean: Vector((153.20001220703125, 1007.7667236328125, 12.000000953674316))


## Save to file

In [60]:
for i in range(len(vectors)):
    v1r = vectors[i].copy()
    
    v1r.rotate(qm)
    v1r += trans_vec

    # Update in df
    df.at[i, 'Axis 0'] = round(v1r.x)
    df.at[i, 'Axis 1'] = round(v1r.y)
    df.at[i, 'Axis 2'] = round(v1r.z)

In [61]:
# Save new file
df.to_csv(csv_path.replace('.csv', '_rotated_translated.csv'), index=False)